## Introduction

Onderstraand vindt u een dataset gericht op de 'attrition' van werknemers. Met attrition wordt de slijtage van werknemers bedoeld. Hiermee wordt bedoeld dat het aantal werknemers afneemt en niet wordt vervangen. Er zijn veel vairabelen die hieraan bijdragen. Dit kan te maken hebben met slechte omstandigheden, geen groeimogelijkheden, ziekte en inkomen. 

Voor deze opdracht worden de volgende variabelen meegenomen om te voorspellen of iemand het bedrijf verlaat en dus 'yes' scoort op 'attrition':
- Age
- Gender
- JobSatisfaction
- MonthlyIncome
- EnvironmentSatisfaction
- TotalWorkingYears
- YearsSinceLastPromotion
- RelationshipSatisfaction

Bovenstaande variabelen worden meegenomen in het onderzoek aangezien wordt verwacht dat deze variabelen van grote invloed zijn op 'attrition'. 

## Importeren libraries

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Importeren van dataset

In [2]:
df1 = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df1.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Data Cleaning

#### Er wordt een subset gemaakt van de gekozen variabelen om vervolgens te kunnen gebruiken in het verdere onderzoek. De overige variabelen worden uitgesloten.

In [3]:
df1 = df1[['Attrition','Age','Gender','EnvironmentSatisfaction','JobSatisfaction','MonthlyIncome','RelationshipSatisfaction','TotalWorkingYears','YearsSinceLastPromotion']]
df1.head()

,Attrition,Age,Gender,EnvironmentSatisfaction,JobSatisfaction,MonthlyIncome,RelationshipSatisfaction,TotalWorkingYears,YearsSinceLastPromotion
0,Yes,41,Female,2,4,5993,1,8,0
1,No,49,Male,3,2,5130,4,10,1
2,Yes,37,Male,4,3,2090,2,7,0
3,No,33,Female,4,3,2909,3,8,3
4,No,27,Male,1,2,3468,4,6,2


Er wordt gekeken naar de subset om NaN waardes en andere verschillen tussen de variabelen te ontdekken. Er blijken geen NaN waardes te zijn. Alle variabelen behalve 'gender' zijn integers. 

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Attrition                 1470 non-null   object
 1   Age                       1470 non-null   int64 
 2   Gender                    1470 non-null   object
 3   EnvironmentSatisfaction   1470 non-null   int64 
 4   JobSatisfaction           1470 non-null   int64 
 5   MonthlyIncome             1470 non-null   int64 
 6   RelationshipSatisfaction  1470 non-null   int64 
 7   TotalWorkingYears         1470 non-null   int64 
 8   YearsSinceLastPromotion   1470 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 103.5+ KB


## Exploratory data analysis
Om relevante grafieken te kunnen laten zien, moeten de categorische variabelen eerst nummeriek worden. Daarom worden de kolommen 'gender' en 'attrition' onderstaand omgevormd van een object tot een integer. 

In [7]:
df1.Attrition.value_counts(), df1.Gender.value_counts()
df1_dummies = pd.get_dummies(df1,columns=['Gender','Attrition'],drop_first= True)
df1_dummies.head()

,Age,EnvironmentSatisfaction,JobSatisfaction,MonthlyIncome,RelationshipSatisfaction,TotalWorkingYears,YearsSinceLastPromotion,Gender_Male,Attrition_Yes
0,41,2,4,5993,1,8,0,0,1
1,49,3,2,5130,4,10,1,1,0
2,37,4,3,2090,2,7,0,1,1
3,33,4,3,2909,3,8,3,0,0
4,27,1,2,3468,4,6,2,1,0


Er is gekozen om meer inzicht te bieden aan de hand van een tabel aangezien grafieken u niet tot meer inzicht zouden brengen. 

In [14]:
df1_dummies.corr()

,Age,EnvironmentSatisfaction,JobSatisfaction,MonthlyIncome,RelationshipSatisfaction,TotalWorkingYears,YearsSinceLastPromotion,Gender_Male,Attrition_Yes
Age,1.000000,0.010146,-0.004892,0.497855,0.053535,0.680381,0.216513,-0.036311,-0.159205
EnvironmentSatisfaction,0.010146,1.000000,-0.006784,-0.006259,0.007665,-0.002693,0.016194,0.000508,-0.103369
JobSatisfaction,-0.004892,-0.006784,1.000000,-0.007157,-0.012454,-0.020185,-0.018214,0.033252,-0.103481
MonthlyIncome,0.497855,-0.006259,-0.007157,1.000000,0.025873,0.772893,0.344978,-0.031858,-0.159840
RelationshipSatisfaction,0.053535,0.007665,-0.012454,0.025873,1.000000,0.024054,0.033493,0.022868,-0.045872
TotalWorkingYears,0.680381,-0.002693,-0.020185,0.772893,0.024054,1.000000,0.404858,-0.046881,-0.171063
YearsSinceLastPromotion,0.216513,0.016194,-0.018214,0.344978,0.033493,0.404858,1.000000,-0.026985,-0.033019
Gender_Male,-0.036311,0.000508,0.033252,-0.031858,0.022868,-0.046881,-0.026985,1.000000,0.029453
Attrition_Yes,-0.159205,-0.103369,-0.103481,-0.159840,-0.045872,-0.171063,-0.033019,0.029453,1.000000


Op basis van bovenstaande tabel valt te concluderen dat de gekozen variabelen niet veel invloed hebben op 'attrition'. Er valt wel te constateren dat leeftijd, inkomen, werkjaren en promotie met elkaar correleren, maar 'attrition' valt hier niet onder. 'Attrition' correleert het meeste met 'Gender_Male'. 